In [25]:
import numpy as np
import pandas as pd
import pymysql as mysql

In [35]:
connection = mysql.connect(host='localhost',
                           user='musicx',
                           password='Eric2mysql',
                           db='mx_stock',
                           charset='utf8mb4',
                           cursorclass=mysql.cursors.DictCursor)


query_sql = 'select * from high_mmt where datediff(current_date(), `date`) < 30 and `level`=2'
industries = pd.read_sql_query(query_sql, connection)
industries

,id,industry,high,full,rate,score,date,level
0,5984,食品饮料-食品加工,32,57,0.5614,17.9649,2020-08-31,2
1,5985,机械设备-通用机械,27,114,0.2368,6.3947,2020-08-31,2
2,5986,汽车-汽车零部件,29,133,0.2180,6.3233,2020-08-31,2
3,5987,化工-化学制品,37,222,0.1667,6.1667,2020-08-31,2
4,5988,机械设备-专用设备,32,179,0.1788,5.7207,2020-08-31,2
...,...,...,...,...,...,...,...,...
1746,17695,商业贸易-贸易,1,22,0.0455,0.0455,2020-09-29,2
1747,17696,医药生物-医疗服务,1,24,0.0417,0.0417,2020-09-29,2
1748,17697,医药生物-医药商业,1,25,0.0400,0.0400,2020-09-29,2
1749,17698,综合-综合,1,32,0.0313,0.0313,2020-09-29,2


In [45]:
industries['rank'] = industries.groupby('date')['score'].rank(ascending=False, na_option='bottom')
industries['cap'] = industries.apply(lambda r: '{}/{}'.format(r.high, r.full), axis=1)
industries

,id,industry,high,full,rate,score,date,level,rank,cap
0,5984,食品饮料-食品加工,32,57,0.5614,17.9649,2020-08-31,2,1.0,32/57
1,5985,机械设备-通用机械,27,114,0.2368,6.3947,2020-08-31,2,2.0,27/114
2,5986,汽车-汽车零部件,29,133,0.2180,6.3233,2020-08-31,2,3.0,29/133
3,5987,化工-化学制品,37,222,0.1667,6.1667,2020-08-31,2,4.0,37/222
4,5988,机械设备-专用设备,32,179,0.1788,5.7207,2020-08-31,2,5.0,32/179
...,...,...,...,...,...,...,...,...,...,...
1746,17695,商业贸易-贸易,1,22,0.0455,0.0455,2020-09-29,2,68.0,1/22
1747,17696,医药生物-医疗服务,1,24,0.0417,0.0417,2020-09-29,2,69.0,1/24
1748,17697,医药生物-医药商业,1,25,0.0400,0.0400,2020-09-29,2,70.0,1/25
1749,17698,综合-综合,1,32,0.0313,0.0313,2020-09-29,2,71.0,1/32


In [46]:
ind_pivot = industries.pivot(index='industry', columns='date', values=['score','rank','cap'])
ind_pivot = ind_pivot.reindex(sorted(ind_pivot.columns, reverse=True), axis=1)
ind_pivot.fillna(0, inplace=True)
ind_pivot.sort_values(by=ind_pivot.columns[0], ascending=False, inplace=True)

In [47]:
pd.options.display.max_rows = 1000
top_ind = ind_pivot.loc[(ind_pivot.score > 1).any(axis=1),:]
top_ind

score                                              \
date          2020-09-29 2020-09-28 2020-09-25 2020-09-23 2020-09-22   
industry                                                               
电气设备-电源设备         7.6800    13.6533    10.4533    13.6533     8.2237   
化工-化学制品           3.8054     5.8643     6.8824     8.3665     6.8514   
机械设备-通用机械         3.1667     6.3947    10.7456    12.0088     9.5526   
计算机-计算机应用         2.5474     3.8368     5.3895     6.0842     5.0847   
机械设备-专用设备         2.4500     4.3556     5.0279     4.3799     5.3389   
汽车-汽车整车           2.2273     5.5000     4.5455     5.5000     4.5455   
电子-半导体            1.8519     3.6296     3.1296     2.2407     1.5000   
国防军工-航空装备         1.5806     1.5806     2.0645     3.9032     3.2258   
休闲服务-旅游综合         1.3333     1.3333     1.3333     1.3333     0.7500   
化工-橡胶             1.3158     0.8421     1.3158     1.3158     1.3158   
电子-元件             1.1951     1.1951     1.9756     1.5610     1.1951   
采掘-煤炭开采           0.9474     5.9211     1.2895     0.1053     0.6579   
纺织服装-服装家纺         0.9423     2.3269     3.7692     1.5577     2.2830   
汽车-汽车零部件          0.9030     4.2985     5.0448     5.4812     5.8947   
建筑装饰-园林工程         0.7273     1.1364     1.6364     0.4091     0.7273   
传媒-营销传播           0.7143     2.8571     1.8824     1.8824     3.0303   
国防军工-地面兵装         0.6429     0.6429     1.1429     1.7857     0.6429   
建筑材料-其他建材         0.6098     1.5610     1.5610     1.1951     1.1951   
交通运输-物流           0.6098     1.5610     0.8780     0.6098     1.5610   
食品饮料-饮料制造         0.6098     0.6098     0.6098     0.8780     1.1951   
非银金融-保险           0.5714     2.2857     0.5714     0.1429     0.0000   
电气设备-电气自动化设备      0.5682     2.7500     2.7500     1.8409     1.8409   
电子-电子制造           0.5373     1.2090     0.7313     0.7313     0.5373   
机械设备-金属制品         0.5294     2.1176     2.1176     1.4706     1.4706   
商业贸易-一般零售         0.5102     2.0408     1.3061     0.7347     0.5102   
交通运输-航空运输         0.5000     3.1250     0.5000     1.1250     1.1250   
通信-通信设备           0.4900     1.0000     1.2100     0.6400     1.0000   
电子-光学光电子          0.4800     1.3333     1.6133     2.2533     2.2533   
电气设备-高低压设备        0.3906     1.5625     1.8906     1.5625     2.6406   
医药生物-中药           0.3676     0.9412     1.4706     0.9412     1.1912   
医药生物-化学制药         0.3364     0.5981     1.1308     0.9346     1.1308   
家用电器-白色家电         0.3265     1.3061     1.6531     2.0408     2.0408   
化工-塑料             0.2813     1.1250     1.1250     2.0000     2.0000   
机械设备-仪器仪表         0.2647     0.4706     1.0588     2.9412     2.3824   
有色金属-金属非金属新材料     0.2571     1.0286     1.0286     1.8286     0.4706   
轻工制造-家用轻工         0.2222     1.3889     1.1250     0.6806     0.6806   
房地产-房地产开发         0.2137     1.6752     0.6923     1.2308     1.9231   
化工-化学纤维           0.1739     0.3913     0.3913     0.3913     0.6957   
公用事业-环保工程及服务      0.1698     0.9245     0.4717     0.4717     1.2308   
非银金融-多元金融         0.1379     0.5517     0.5517     0.5517     0.3103   
电子-其他电子           0.1333     0.5333     0.5333     0.5333     1.1613   
传媒-文化传媒           0.1200     0.4800     0.8533     0.4800     0.6533   
钢铁-钢铁             0.1176     1.4412     0.2647     0.2647     0.2647   
休闲服务-景点           0.1000     0.1000     1.6000     0.1000     0.4000   
非银金融-证券           0.0870     1.3913     1.3913     0.0217     0.3478   
有色金属-工业金属         0.0769     0.3077     0.4808     1.2308     0.9423   
传媒-互联网传媒          0.0667     0.2667     0.1500     0.8167     0.6000   
农林牧渔-农产品加工        0.0588     0.0588     0.0588     0.0588     0.0588   
建筑装饰-房屋建设         0.0500     0.0500     0.0500     0.2000     0.2105   
商业贸易-贸易           0.0455     0.4091     0.1818     0.7273     1.1364   
医药生物-医疗服务         0.0417     1.0417     0.6667     0.3750     0.1667   
医药生物-医药商业         0.0400     0.3600     0.1600     0.3600     0.0000   
综合-综合             0.0313     0.2813     1.1250     0.7813  

In [48]:
top_ind.to_excel('top_rps_blocks.xls')

In [40]:
query_sql = 'select * from high_mmt where datediff(current_date(), `date`) < 8 and `level`=3'
sub_industries = pd.read_sql_query(query_sql, connection)
sub_industries

,id,industry,high,full,rate,score,date,level
0,16227,汽车-汽车零部件-汽车零部件,28,133,0.2105,5.8947,2020-09-22,3
1,16228,机械设备-通用机械-机械基础件,13,39,0.3333,4.3333,2020-09-22,3
2,16229,化工-化学制品-其他化学制品,21,105,0.2000,4.2000,2020-09-22,3
3,16230,电气设备-电源设备-光伏设备,11,29,0.3793,4.1724,2020-09-22,3
4,16231,电气设备-电气自动化设备-工控自动化,8,17,0.4706,3.7647,2020-09-22,3
5,16232,机械设备-专用设备-其它专用机械,18,96,0.1875,3.3750,2020-09-22,3
6,16233,国防军工-航空装备-航空装备,10,31,0.3226,3.2258,2020-09-22,3
7,16234,计算机-计算机应用-IT服务,18,104,0.1731,3.1154,2020-09-22,3
8,16235,传媒-营销传播-营销服务,10,33,0.3030,3.0303,2020-09-22,3
9,16236,汽车-汽车整车-乘用车,6,12,0.5000,3.0000,2020-09-22,3


In [41]:
sub_industries['rank'] = sub_industries.groupby('date')['score'].rank(ascending=False, na_option='bottom')
sub_industries

,id,industry,high,full,rate,score,date,level,rank
0,16227,汽车-汽车零部件-汽车零部件,28,133,0.2105,5.8947,2020-09-22,3,1.0
1,16228,机械设备-通用机械-机械基础件,13,39,0.3333,4.3333,2020-09-22,3,2.0
2,16229,化工-化学制品-其他化学制品,21,105,0.2000,4.2000,2020-09-22,3,3.0
3,16230,电气设备-电源设备-光伏设备,11,29,0.3793,4.1724,2020-09-22,3,4.0
4,16231,电气设备-电气自动化设备-工控自动化,8,17,0.4706,3.7647,2020-09-22,3,5.0
5,16232,机械设备-专用设备-其它专用机械,18,96,0.1875,3.3750,2020-09-22,3,6.0
6,16233,国防军工-航空装备-航空装备,10,31,0.3226,3.2258,2020-09-22,3,7.0
7,16234,计算机-计算机应用-IT服务,18,104,0.1731,3.1154,2020-09-22,3,8.0
8,16235,传媒-营销传播-营销服务,10,33,0.3030,3.0303,2020-09-22,3,9.0
9,16236,汽车-汽车整车-乘用车,6,12,0.5000,3.0000,2020-09-22,3,10.0


In [43]:
sub_ind_pivot = sub_industries.pivot(index='industry', columns='date', values=['score','rank'])
sub_ind_pivot = sub_ind_pivot.reindex(sorted(sub_ind_pivot.columns, reverse=True), axis=1)
sub_ind_pivot.fillna(0, inplace=True)
sub_ind_pivot.sort_values(by=sub_ind_pivot.columns[0], ascending=False, inplace=True)
sub_ind_pivot.loc[(sub_ind_pivot.score > 1).any(axis=1),:]

score                                              \
date                 2020-09-29 2020-09-28 2020-09-25 2020-09-23 2020-09-22   
industry                                                                      
电气设备-电源设备-光伏设备           5.8276     9.9655     9.9655     7.7586     4.1724   
电气设备-电源设备-风电设备           2.7692     3.7692     1.2308     2.7692     1.9231   
化工-化学制品-其他化学制品           2.4615     3.4712     4.2404     4.6538     4.2000   
计算机-计算机应用-IT服务           1.6095     2.1429     4.2000     3.4381     3.1154   
国防军工-航空装备-航空装备           1.5806     1.5806     2.0645     3.9032     3.2258   
电子-半导体-分立器件              1.5000     1.5000     1.5000     1.5000     1.5000   
机械设备-通用机械-其它通用机械         1.4000     2.3143     2.8571     3.4571     2.3143   
化工-塑料-合成革                1.3333     1.3333     1.3333     3.0000     1.3333   
休闲服务-旅游综合-旅游综合           1.3333     1.3333     1.3333     1.3333     0.7500   
有色金属-稀有金属-锂              1.3333     1.3333     0.3333     1.3333     0.0000   
化工-橡胶-炭黑                 1.3333     1.3333     1.3333     1.3333     1.3333   
电子-半导体-集成电路              1.2895     2.6316     2.1316     1.2895     0.6579   
纺织服装-服装家纺-鞋帽             1.2857     1.2857     1.2857     0.5714     0.5714   
食品饮料-饮料制造-啤酒             1.2857     0.5714     0.5714     1.2857     1.2857   
机械设备-专用设备-其它专用机械         1.2604     2.3438     3.0421     3.0421     3.3750   
采掘-煤炭开采-煤炭开采             1.2414     7.7586     1.6897     0.1379     0.8621   
机械设备-通用机械-磨具磨料           1.2308     2.7692     4.9231     3.7692     1.9231   
化工-橡胶-轮胎                 1.1250     0.5000     0.5000     0.5000     0.5000   
商业贸易-一般零售-多业态零售          1.0667     2.4000     1.6667     1.0667     0.6000   
计算机-计算机应用-软件开发           0.9529     1.6941     1.4235     2.6471     1.9882   
电气设备-电气自动化设备-工控自动化       0.9412     2.8824     2.8824     2.1176     3.7647   
电子-元件-被动元件               0.9412     0.9412     2.1176     2.1176     1.4706   
汽车-汽车零部件-汽车零部件           0.9030     4.2985     5.0448     5.4812     5.8947   
建筑材料-其他建材-耐火材料           0.8000     1.8000     0.8000     0.2000     0.2000   
汽车-汽车整车-乘用车              0.7500     3.0000     3.0000     3.0000     3.0000   
建筑装饰-园林工程-园林工程           0.7273     1.1364     1.6364     0.4091     0.7273   
传媒-营销传播-营销服务             0.7143     2.8571     1.8824     1.8824     3.0303   
汽车-汽车整车-商用载客车            0.6667     1.5000     0.6667     1.5000     1.5000   
国防军工-地面兵装-地面兵装           0.6429     0.6429     1.1429     1.7857     0.6429   
化工-化学制品-涂料油漆油墨制造         0.6429     1.7857     1.1429     2.5714     1.7857   
交通运输-物流-物流               0.6098     1.5610     0.8780     0.6098     1.5610   
非银金融-保险-保险               0.5714     2.2857     0.5714     0.1429     0.0000   
机械设备-金属制品-金属制品           0.5294     2.1176     2.1176     1.4706     1.4706   
电子-光学光电子-LED             0.5000     1.5313     1.5313     3.1250     2.5313   
交通运输-航空运输-航空运输           0.5000     3.1250     0.5000     1.1250     1.1250   
机械设备-专用设备-印刷包装机械         0.4444     0.4444     0.4444     1.0000     1.7778   
机械设备-通用机械-机械基础件          0.4103     1.2564     2.5641     4.3333     4.3333   
电气设备-电源设备-储能设备           0.4091     1.6364     1.1364     2.2273     1.6364   
医药生物-中药-中药               0.3676     0.9412     1.4706     0.9412     1.1912   
机械设备-专用设备-工程机械           0.3333     0.7500     1.3333     1.3333     1.3333   
医药生物-化学制药-化学制剂           0.3205     0.6282     1.0385     1.0385     1.0385   
机械设备-仪器仪表-仪器仪表           0.2647     0.4706     1.0588     2.9412     2.3824   
纺织服装-服装家纺-其他服装           0.2353     0.9412     1.4706     0.5294     0.5294   
房地产-房地产开发-房地产开发          0.2137     1.6752     0.6923     1.2308     1.9231   
电气设备-电源设备-其它电源设备         0.2000     0.2000     0.2000     1.8000     0.6667   
钢铁-钢铁-普钢                 0.1818     1.1364     0.1818     0.1818     0.1818   
公用事业-环保工程及服务-环保工程及服务     0.1698     0.9245     0.4717     0.4717     1.2308   
机械设备-通用机械-内燃机            0.1667     0.6667     2.6667     1.5000     1.5000 

In [34]:
industries.to_excel('rps_industry_raw.xls')